In [ ]:
# imports

# external modules
import sys
import os
import numpy as np
import pandas as pd
import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
# framework modules
sys.path.append('../')
import plotting.plottools
importlib.reload(plotting.plottools)
from plotting.plottools import plot_histogram
import training.prepare_training_set
importlib.reload(training.prepare_training_set)
from training.prepare_training_set import prepare_training_data_from_files
# local modules
import fake_anomaly
importlib.reload(fake_anomaly)

In [ ]:
# load the training set

files = ([
    '../data/data/ZeroBias-Run2023C-PromptReco-v1-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1_preprocessed.parquet',
    #'../data/data/ZeroBias-Run2023C-PromptReco-v2-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1_preprocessed.parquet',
    #'../data/data/ZeroBias-Run2023C-PromptReco-v3-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1_preprocessed.parquet',
    #'../data/data/ZeroBias-Run2023C-PromptReco-v4-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1_preprocessed.parquet',
    #'../data/data/ZeroBias-Run2023D-PromptReco-v1-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1_preprocessed.parquet',
    #'../data/data/ZeroBias-Run2023D-PromptReco-v2-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1_preprocessed.parquet',
])
kwargs = ({
    'verbose': True,
    'entries_threshold': 10000,
    'skip_first_lumisections': 5
})
(eval_data, eval_runs, eval_lumis) = prepare_training_data_from_files(files, **kwargs)

In [ ]:
# make a mask where values are often zero

shape_mask = (np.sum(eval_data[:,:,:,0]==0, axis=0)>len(eval_data)/2.)

fig,ax = plt.subplots()
plot_histogram(shape_mask, fig=fig, ax=ax, caxrange=(-0.01,1))
ax.text(0.02, 1.02, 'Shape mask', transform=ax.transAxes, fontsize=12)

In [ ]:
# load keras model

modelname = '../models/model_20231115_Run2023C-v1_PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1.keras'
model = keras.models.load_model(modelname)

In [ ]:
# load average occupancy or error of training set

avgresponsename = '../models/model_20231115_Run2023C-v1_PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1_avgoccupancy.npy'
avgresponse = np.load(avgresponsename)
avgresponse = np.square(avgresponse)

fig,ax = plt.subplots()
plot_histogram(avgresponse, fig=fig, ax=ax)
ax.text(0.02, 1.02, 'Average response on training set', transform=ax.transAxes, fontsize=12)
avgresponse[avgresponse==0] = 1
avgresponse = np.expand_dims(avgresponse, axis=2)

In [ ]:
# make a collection of fake anomalies

nanomalies = 100
random_indices = np.random.choice(len(eval_data), size=nanomalies, replace=False)

anomalies = np.zeros((nanomalies, eval_data.shape[1], eval_data.shape[2], 1))
params = []
for newidx,origidx in enumerate(random_indices):
    hist = eval_data[origidx,:,:,0]
    (anomalous_hist, paramdict) = fake_anomaly.dead_rectangle(hist, shape_mask=~shape_mask)
    #(anomalous_hist, paramdict) = fake_anomaly.dead_sector(hist)
    #(anomalous_hist, paramdict) = fake_anomaly.hot_rectangle(hist, shape_mask=~shape_mask)
    #(anomalous_hist, paramdict) = fake_anomaly.hot_sector(hist)
    anomalies[newidx,:,:,0] = anomalous_hist
    params.append(paramdict)

In [ ]:
# evaluate the model

predictions = model.predict(eval_data)
predictions[predictions<0] = 0.
predictions[:,shape_mask] = 0.

predictions_anomalies = model.predict(anomalies)
predictions_anomalies[predictions_anomalies<0] = 0.
predictions_anomalies[:,shape_mask] = 0.

In [ ]:
# calculate squared difference
errors = np.square(eval_data - predictions)
errors_anomalies = np.square(anomalies - predictions_anomalies)

# space correction
errors_corrected = errors/avgresponse
errors_anomalies_corrected = errors_anomalies/avgresponse

In [ ]:
# make plots

nplots = 5
plotids = np.random.choice(len(anomalies), size=nplots, replace=False)

for i in plotids:
    fig,axs = plt.subplots(figsize=(30,6), ncols=4)
    plot_histogram(anomalies[i,:,:,0], fig=fig, ax=axs[0])
    plot_histogram(predictions_anomalies[i,:,:,0], fig=fig, ax=axs[1])
    plot_histogram(errors_anomalies[i,:,:,0], fig=fig, ax=axs[2], caxrange=(-0.001, 0.01))
    plot_histogram(errors_anomalies_corrected[i,:,:,0], fig=fig, ax=axs[3], caxrange=(-0.01, 0.5))
    axs[0].text(0.02, 1.02, 'Fake anomaly', transform=axs[0].transAxes, fontsize=12)